# LBM Step 4: Collate Planes

In [8]:
import sys
from pathlib import Path
import os
import numpy as np
import zarr
import napari

import logging
import matplotlib.pyplot as plt

# Importing this WONT WORK withotu adding to the path
sys.path.append('../..')  # TODO: Take this out when we upload to pypi
import lbm_caiman_python as lcp

import mesmerize_core as mc
from mesmerize_core.caiman_extensions.cnmf import cnmf_cache

try:
    import cv2
    cv2.setNumThreads(0)
except():
    pass

logging.basicConfig()

os.environ["CONDA_PREFIX_1"] = ""
if os.name == "nt":
    # disable the cache on windows, this will be automatic in a future version
    cnmf_cache.set_maxsize(0)


In [9]:
parent_path = Path().home() / "caiman_data" / 'animal_01' / "session_01"

batch_path = parent_path / 'batch.pickle'
mc.set_parent_raw_data_path(str(parent_path))

# create a new batch
try:
    df = mc.load_batch(batch_path)
except (IsADirectoryError, FileNotFoundError):
    df = mc.create_batch(batch_path)

df=df.caiman.reload_from_disk()
df

,algo,item_name,input_movie_path,params,outputs,added_time,ran_time,algo_duration,comments,uuid
0,mcorr,extracted_plane_1,tiff\extracted_plane_1.tif,"{'main': {'var_name_hdf5': 'mov', 'max_shifts'...",{'mean-projection-path': b32f41bf-a9a5-4965-be...,2024-09-26T11:56:54,2024-09-26T12:02:55,77.3 sec,None,b32f41bf-a9a5-4965-be7c-e6779e854328
1,cnmf,cnmf_1,b32f41bf-a9a5-4965-be7c-e6779e854328\b32f41bf-...,"{'main': {'fr': 9.62, 'dxy': (1.0, 1.0), 'deca...",{'mean-projection-path': a057e39e-a2df-41d3-82...,2024-09-26T16:26:20,2024-09-26T16:28:48,143.18 sec,None,a057e39e-a2df-41d3-8217-83c9cd7ffb6d
2,cnmf,extracted_plane_1,b32f41bf-a9a5-4965-be7c-e6779e854328\b32f41bf-...,"{'main': {'fr': 9.62, 'dxy': (1.0, 1.0), 'deca...",{'mean-projection-path': 5cb543ec-5358-4b35-83...,2024-09-27T14:48:59,2024-09-27T14:50:30,80.14 sec,None,5cb543ec-5358-4b35-83cf-bfd19fa06a68
3,cnmf,extracted_plane_1,b32f41bf-a9a5-4965-be7c-e6779e854328\b32f41bf-...,"{'main': {'fr': 9.62, 'dxy': (1.0, 1.0), 'deca...",{'mean-projection-path': df52a609-d986-4bc0-a0...,2024-09-27T14:48:59,2024-09-27T14:51:46,75.93 sec,None,df52a609-d986-4bc0-a0a2-526cbfda057d
4,cnmf,extracted_plane_1,b32f41bf-a9a5-4965-be7c-e6779e854328\b32f41bf-...,"{'main': {'fr': 9.62, 'dxy': (1.0, 1.0), 'deca...",{'mean-projection-path': b68a8efe-1a3b-4392-b6...,2024-09-27T14:48:59,2024-09-30T10:30:15,96.62 sec,None,b68a8efe-1a3b-4392-b6e7-416312f1f945
5,cnmf,extracted_plane_1,b32f41bf-a9a5-4965-be7c-e6779e854328\b32f41bf-...,"{'main': {'fr': 9.62, 'dxy': (1.0, 1.0), 'deca...",{'mean-projection-path': 15e6fb81-fac4-457b-96...,2024-09-27T14:48:59,2024-09-30T11:32:09,89.61 sec,None,15e6fb81-fac4-457b-96da-dbb8e2d7b669
6,cnmf,k=650,b32f41bf-a9a5-4965-be7c-e6779e854328\b32f41bf-...,"{'main': {'fr': 9.62, 'dxy': (1.0, 1.0), 'deca...",None,2024-09-30T11:30:32,None,None,None,ef3ef585-a571-45cd-ab1b-3a82731f1b0e


In [15]:
traces = df.iloc[-2].cnmf.get_temporal()
traces.shape

nt = []
for i, row in df.iterrows():
    if row["outputs"] is not None and row['algo']=='cnmf': # item has already been run
        nt.append(df.iloc[i].cnmf.get_temporal())

[len(n) for n in (nt)]

[1314, 188, 208, 357, 413]